In [1]:
'''
45325919, Anthony Cerqui
https://github.com/littletuna4/bism3206-assign
'''

In [2]:
#Use this cell to import all the required libraries.
# %pip install gensim colorama nltk scikit-learn matplotlib seaborn pandas numpy scipy<1.13 ipykernel openpyxl


# Core
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# NLP
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import gensim
import gensim.corpora as corpora
from gensim.models.coherencemodel import CoherenceModel
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.neural_network import MLPClassifier

# Flavour
from colorama import Fore, Style

In [3]:
#your work should only start with the following code
df=pd.read_excel("Assignment_Data.xlsx")

#### <font color="red"> When you are preparing your assignment, you can use as many cells as you want. However, before submitting, please ensure that you consolidate all the necessary codes for each section into a single cell according to the template requirements. Failing to follow the template will lead to a loss of marks.

# Data exploration

_In this cell, in no more than 200 words, you can elaborate on the major understanding of data based on your data exploration._

The dataset relates to social media data, containing information about news authors, headlines, tweets, and various metrics about user engagement and demographic details. It has a total of 134,198 entries across 16 features, without duplicate entries, indicating good data cleanliness in terms of redundancy. Several features have a significant amount of missing data, particularly user_following and user_country, which could impact analysis related to user demographics and behavior.

The descriptive statistics reveal a highly skewed distribution in engagement metrics (post_replies, post_retweets, post_favourites, post_quotes), with most posts receiving zero engagement but a few outliers receiving extremely high interaction, suggesting viral behavior for a minority of posts. User metrics like user_followers, user_favourites, and user_tweets_count also display wide variances, indicating a diverse user base from minimally active to influential users.

Misinformation is flagged in roughly half of the entries (mean=0.514), highlighting the dataset's utility for studying the spread and impact of misinformation across social platforms. The data includes a variety of unique authors, headlines, and tweets, which can be useful for textual analysis and pattern recognition in social media discussions. The significant presence of missing values, especially in demographic-related columns, suggests a need for careful handling and possibly imputation techniques for more robust analyses.

In [4]:
#Use this cell for data exploration.

def deco(a_string):
  return Style.BRIGHT+Fore.BLUE+a_string+Style.RESET_ALL


  
print(
    deco("df.head()\n"),df.head(),
    deco("df.info()\n"),df.info(),
    deco("df.describe()\n"),df.describe(),
    deco("df.columns\n"),df.columns,
    deco("df.shape\n"),df.shape,
    deco("df.isnull().sum()\n"),df.isnull().sum(),
    deco("df.duplicated().sum()\n"),df.duplicated().sum(),
    deco("df.nunique()\n"),df.nunique(),
    sep="\n"
)



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134198 entries, 0 to 134197
Data columns (total 16 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   news_author             134198 non-null  object 
 1   news_headline           134130 non-null  object 
 2   related_tweet           134150 non-null  object 
 3   post_replies            134175 non-null  float64
 4   post_retweets           134198 non-null  int64  
 5   post_favourites         134198 non-null  int64  
 6   post_quotes             134198 non-null  int64  
 7   user_followers          134170 non-null  float64
 8   user_following          44732 non-null   float64
 9   user_friends            134103 non-null  float64
 10  user_mentions           134143 non-null  float64
 11  user_favourites         134170 non-null  float64
 12  user_tweets_count       134166 non-null  float64
 13  user_tweet_count_lists  134179 non-null  float64
 14  user_country        

# Text preparation

In this cell, in no more than 150 words, you can elaborate on the the innovativeness and depth of your work on text preparation.

In [5]:
#Use this cell for text preparation.

# Tokenise the words using punkt
nltk.download('punkt')

'''
Lowercasing:
We can remove the case sensitivity of the text by converting all the words to lowercase.
'''

'''
Stopword removal
We can remove unimportant words to reduce the computation load and improve the potency of the training data
'''

'''
Punctuation and special character removal
Much like stopword removal, this technique is good for omitting unimportant data.
'''

'''
stemming
Stemming is a technique used to reduce words to their root form.
'''

'''
lemmatization
lemmatization is another technique used to reduce words to their root form.
'''

'''
Bag Of words
The bag of words model is a simple model that represents text data as a bag of words.
It ignores order and grammar, and only considers the frequency of words in the text - it also serves as
the input to TLF-IDF and Word2Vec models.
'''

'''
TFIDF -
TFIDF is a measure of how important a word is in a document.
It is calculated by multiplying the term frequency of a word in a document by the
inverse document frequency of the word across all documents.
'''

'''
Dimensionality Reduction
Dimensionality reduction is a technique used to reduce the number of features in a dataset.
This reduces the computational load at the expense of some information loss.
'''



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\seems\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

# Feature engineering

In this cell, in no more than 150 words, you can elaborate on the the innovativeness of your work on feature generation.

##### Feature 1 generation

In [11]:
#Use this cell for text feature_1 generation.

'''
Vadar sentiment

'''


'''
LDA (Latent Dirichlet Allocation)

'''


'''
Descision Tree/Random forest

'''

'''
POS Tagging
Considers Grammatical features of words
'''

.
.
.

##### Feature N generation

In [13]:
#Use this cell for text feature_N generation.

# Model Development
#### <font color="red">You must remove this section from your notebook before submission
<font color="black">This section is for you to work and develop any models you choose. <br>When you have found the best performing model, you should move it to the **Model building & evaluation** section below.

In [11]:
#Use this cell for developing or evaluating model_1.

.
.
.

In [ ]:
#Use this cell for developing or evaluating model_N.

# Model building & evaluation
Once you have found the most optimal model, you should only show the model building and evaluation here!

In this cell, in no more than 300 words, you can elaborate on development and performance of your best model.

In [ ]:
#Use this cell for the best models building & evaluation.